# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902045


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:  10%|█         | 3/30 [00:05<00:49,  1.82s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:34,  1.35s/it]

Rendering models:  17%|█▋        | 5/30 [00:05<00:25,  1.01s/it]

Rendering models:  20%|██        | 6/30 [00:06<00:18,  1.32it/s]

...

Rendering models:  90%|█████████ | 27/30 [00:11<00:00,  4.44it/s]

Rendering models:  93%|█████████▎| 28/30 [00:11<00:00,  4.50it/s]

Rendering models:  97%|█████████▋| 29/30 [00:12<00:00,  4.60it/s]

Rendering models: 100%|██████████| 30/30 [00:12<00:00,  4.64it/s]

Its_Foster                            0.002301
equidad1                              0.067099
nickoftona                            0.000318
ElisabethB                            0.000285
Thedunkmasta                          0.000989
OregonHiker                           0.000273
BirdLeaf                              0.000315
markbouslog                           0.005786
lovestvck                             0.001929
Alexander00                           0.000324
lsautter                              0.000280
not-logged-in-a62efc8bbed690435003    0.000944
Jessica_Korkmaz                       0.001676
OmGupta                               0.000323
not-logged-in-9a45911eb71184fe7c7f    0.000408
Josel                                 0.000877
peakscience7                          0.000310
stansall                              0.000349
Lavadude                              0.284197
Kuchy                                 0.009378
not-logged-in-a4bed12d9111c0c34e9e    0.004099
ts135992     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())